In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = FetchPairDetails().apply(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:61,383,718 tx:0xe7cf406b670946d98285557885cc1daef1bb9ad12685411eac3f2aaecd93d758
Sync at block:61,383,718 tx:0xfd6a527a3f93984310c34b44a2cc875cc3ca3f5efcbcfa71bf0427c14b4ba97e
Sync at block:61,383,719 tx:0x8054d4b8a0ed84f99dc11cd4ec8b6d5b3761313d4fb08d4ae5c136e7f489d976
Sync at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Sync at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Sync at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Sync at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Sync at block:61,383,723 tx:0xbc9145c16919ba7f0eada3c717119aa25570d2a65461ea2361f1c93c1a9a6d1e
Sync at block:61,383,723 tx:0xe04b26d41f5195c58e22eeb7dd8bfa1b639f8fa9a3c391c91856de168bcb0770
Sync at block:61,383,727 tx:0xe07df91bda2aafd8fd002fd93427eea75b213ed0e7a9e36020722905f944a443
.


In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0x264e6bc3f95633725658e4d9640f7f7d9100f6ac,0xe7cf406b670946d98285557885cc1daef1bb9ad12685...,61383718,1725384897,{'web3_type': <class 'web3._utils.datatypes.Sy...
1,polygon,uniswapv2pair,sync,sushi,0x1ded21f3a458c0372a76bab630a97004b4f80fc0,0xfd6a527a3f93984310c34b44a2cc875cc3ca3f5efcbc...,61383718,1725384897,{'web3_type': <class 'web3._utils.datatypes.Sy...
2,polygon,uniswapv2pair,sync,sushi,0x6a34bc58416381ad92aaae770256ed09ce2b2ec9,0x8054d4b8a0ed84f99dc11cd4ec8b6d5b3761313d4fb0...,61383719,1725384899,{'web3_type': <class 'web3._utils.datatypes.Sy...
3,polygon,uniswapv2pair,sync,sushi,0x0712323f8451cf7acc1141083baa60cc70dc32a8,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sy...
4,polygon,uniswapv2pair,sync,sushi,0xd6251d670b5fdf270bfea6de285003304dd5d177,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sy...
5,polygon,uniswapv2pair,sync,sushi,0x09b647132a26fb0813257656c165b03f8ee84e54,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sy...
6,polygon,uniswapv2pair,sync,sushi,0x3db7148e24be957a6229404c3f7a5fdc948ae543,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sy...
7,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xbc9145c16919ba7f0eada3c717119aa25570d2a65461...,61383723,1725384907,{'web3_type': <class 'web3._utils.datatypes.Sy...
8,polygon,uniswapv2pair,sync,sushi,0x70294d7aa244bd342c536f9b502152564057162e,0xe04b26d41f5195c58e22eeb7dd8bfa1b639f8fa9a3c3...,61383723,1725384907,{'web3_type': <class 'web3._utils.datatypes.Sy...
9,polygon,uniswapv2pair,sync,sushi,0x07a01030cc9b5379492485ba13b471a28d590d49,0xe07df91bda2aafd8fd002fd93427eea75b213ed0e7a9...,61383727,1725384915,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x264e6bc3f95633725658e4d9640f7f7d9100f6ac',
  'tx_hash': '0xe7cf406b670946d98285557885cc1daef1bb9ad12685411eac3f2aaecd93d758',
  'blk_num': 61383718,
  'timestamp': 1725384897,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0x8A953CfE442c5E8855cc6c61b1293FA648BAE472',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'PolyDoge',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 171279100878081485515362,
   'amount1': 6438320157265020485753516265079,
   'price': 37589642.42723281}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x1ded21f3a458c0372a76bab630a97004b4f80fc0',
  'tx_hash': '0xfd6a527a3f93984310c34b44a2cc875cc3ca3f5efcbcfa71bf0427c14b4ba97e',
  'blk_num': 61383718,
  'timestamp': 1725384897,
  'details': {'we